# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量 ok
按照笔记的方法分别推出各个量

注意时间单位的选取!

## 2. 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
a
n()
E()
e
M
i Omega w

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

import DDS_CMpack as DDS

%matplotlib inline

In [2]:
# astronomy constants
R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

In [3]:
miu_GM = G_graviation*M_earth # 国际单位制

miu_GM

397984800000000.0

In [4]:
# input r0,v0,t0
r0 = np.array([-6840.16579265, -1009.61579285,  3216.48004799])*10**3 # unit= m
r0_norm = np.sqrt(sum([ i*i for i in r0])) # m

v0 = np.array([-1.07349321, -4.73916395, -3.70931952])*10**3 # unit= m/SI
v0_norm = np.sqrt(sum([ i*i for i in v0])) # m/SI

t0 = 121.0217445 # unit = SI from the first data we have
## t0 根据自己的单位去选择。

r0_norm/10**3,v0_norm/10**3

(7625.807237219486, 6.113191794664121)

In [5]:
# get a 

a = 1/(2/r0_norm - v0_norm**2/miu_GM) # unit = m

a/R_earth

0.9322594973289169

In [6]:
# get n\E then a
n = np.sqrt(miu_GM/(a**3))

tan_E = (1 - r0_norm/a)*(a**2*n)/(np.dot(r0,v0))

e = np.sqrt( (1 - r0_norm/a)**2 + (np.dot(r0,v0) / (n*a**2))**2 )
# e = 0.01002

print(1 - r0_norm/a)
print(np.dot(r0,v0) / (n*a**2))

cos_E = (1 - r0_norm/a)/e
sin_E = (np.dot(r0,v0) / (a**2*n))/e

e

-0.28393016270596894
0.004044804134243226


0.2839589719215126

In [7]:
def arctan2(sinE,cosE):
    if sinE >=0 and cosE >=0: # I
        output = np.arcsin(sinE)
    elif sinE >=0 and cosE <=0: # II
        output = np.pi - np.arcsin(sinE)
    elif sinE <=0 and cosE >=0: # IV
        output = 2*np.pi + np.arcsin(sinE)
    else: # III
        output = np.pi - np.arcsin(sinE)
    return output


In [8]:
# E1 = np.arcsin(sin_E)
# E2 = np.arccos(cos_E)
# E3 = np.arctan(tan_E)

print(sin_E,cos_E,tan_E)

E = arctan2(sin_E,cos_E)
print(E)
np.degrees(E)

-0.5053327708284814 -0.8629245567990342 1.707636248059451
3.6713602068150015


210.35344492277665

In [9]:
# keplar 定 M
M = E - e*sin_E
M

3.6772767365601564

In [10]:
# 定 i Omega w

#### method 1
P = (cos_E/r0_norm)*r0 - (sin_E/(a*n))*v0
Q = (sin_E/(r0_norm)*np.sqrt(1-e**2))*r0 + ((cos_E - e)/(a*n*np.sqrt(1-e**2)))*v0
R = np.cross(P,Q)

cos_i = R[2]
tan_Omega = -1*R[0]/R[1]

# get w ->[-90,+90]
tan_w = P[2]/Q[2]

############################################################

#### method 2
h = np.cross(r0,v0)
h_norm = np.sqrt(np.dot(h,h))

h_A, h_B, h_C = h[0:3]

# get cos_i, sin_i
cos_i_check = (h_C/h_norm)
tan_i = np.sqrt(h_A**2 + h_B**2) / h_C

sin_i = tan_i*cos_i_check


# get cos_\sin_Omega
sin_Omega = h_A/(h_norm*sin_i)
cos_Omega = -h_B/(h_norm*sin_i)

# finally get i Omega w

i = arctan2(sin_i,cos_i_check)

Omega = arctan2(sin_Omega,cos_Omega)

w = np.arctan(tan_w)

np.degrees(np.array([i,Omega,w]))

array([ 47.36168524,  31.3703739 , -63.49142414])

## 3. 由轨道6根数归算星历表

已知：
a, n(), E(), e, M, [i, Omega, w] = P,Q

t0 = 121.0217445 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [121]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

t = 0.124857*time_unit

Mt = M + n*(t - t0)

print(e*np.sin(Mt))
print(Mt)

-0.005787275935037951
3.6586141786196316


In [122]:
# 迭代 开普勒方程解 E
epsilon = 1e-20
key = 0
En0 = Mt
Enw = [En0]
Nwind = 0
while key == 0:
    Enw.append(Mt + e*np.sin(Enw[Nwind]))

    Nwind += 1
#     print(Enw[Nwind])
    
    delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
#     print(delta_Enw)
#     print('------------')
    
    if delta_Enw < epsilon:
        key = 1;
        
Et = Enw[Nwind]

print('E:',Et)

E: 3.652885305065897


In [123]:
np.cos(i)

0.6773680602089371

In [144]:
# 给出P_new, Q_new
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

print(P,Q)
print(P_new,Q_new)

[ 0.69661115 -0.28522569 -0.65831544] [0.60663306 0.72395805 0.32834701]
[ 0.69663189 -0.28520094 -0.65830421] [0.60667205 0.72397579 0.32834141]


In [145]:
# 归算rt, vt

rt = a*(np.cos(Et) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et)*Q_new
rt_norm = np.sqrt( np.dot(rt,rt) )

vt = -a**2*n/rt_norm*np.sin(Et)*P_new + a**2*n/rt_norm*np.sqrt(1 - e**2)*np.cos(Et)*Q_new
vt_norm = np.sqrt( np.dot(vt,vt) )

L_rt = rt/ np.sqrt( np.dot(rt,rt))

print(rt_norm/10**3,vt_norm/10**3)

rt,vt

7855.60391489427 7.081329396931624


(array([-7095980.17697021,  -794371.19035444,  3275141.61643171]),
 array([-1332.66729448, -5459.24934173, -4308.80732163]))

In [146]:
# 计算赤经赤纬：
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast = P_station/np.sqrt(np.dot(P_station,P_station))

L_forecast

array([-0.92159627,  0.38588585, -0.04186193])

In [83]:
L = [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885]
L = [-0.921558,0.385984,-0.041788]
L

[-0.921558, 0.385984, -0.041788]

In [12]:
np.arange(1,10,0.5)

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5, 7. ,
       7.5, 8. , 8.5, 9. , 9.5])